## This is a pipeline that does all the transformations we are interested in. Just RUN ALL CELLS at once, and wait 2-4 minutes for everything to run. 

### The results are in the bottom cells. First dataframe is the dataframe with all the groupings. The second dataframe has recommendations generated per each search



Imports to make sure all dependencies are installed

In [1]:
#imports
! pip install geopy
from datetime import date
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import geopy.distance
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

This is the pipeline class structure. Don't change it.

In [2]:
# EACH GROOUPING IS A FUNCTION WHICH APPENDS A COLUMN TO THE DATASET


class helperFunctions(object):
    
    
    def group_region_country(self, trips, airport):
        
        """
        This function joins in the trips and airport dataframes 
        in order to enable grouping by region and country
        
        """
        
        
        # LIST OF COLUMNS OF AIRPORTS DATASET
        array = ['location_id', 'location_type', 'airport_group', 'country_code',
               'country', 'region', 'latitude', 'longitude']

        # set up origin and destination alternations
        array_origin = [item + '_origin' for item in array]
        array_destination = [item + '_destination' for item in array]


        # Rename origin airports table version
        rename_origin = {}

        for a, b in zip(array, array_origin):
            rename_origin.update({a:b})

        # rename destination airports table version
        rename_destination = {}

        for a, b in zip(array, array_destination):
            rename_destination.update({a:b})

       
        
        # renaming columns of origin and destination
        airports_origin = airport.rename(columns=rename_origin)
        airports_destination = airport.rename(columns=rename_destination)
        

        # join the 2 dataframes
        joined = pd.merge(trips, airports_origin, how='left', left_on=['origin'], right_on=['location_id_origin']) \
                   .merge(airports_destination, how='left', left_on=['destination'], right_on=['location_id_destination'])

        
        return joined
    
    
    
    
    
    
    # the function the allows me to create bands
    def banding(self, df, categories, colName):
        
        """
        This function groups a column in dataframe df by a certain number of categories
        into percentile cuts
        
        """
    
        # create quartiles (automatic naming depending on size of quartile)
        tiers = pd.qcut(df[colName], q=categories-1, labels=range(1,categories))

        return tiers
    
    
    

    # pass the row in --> distance calculation
    def distance(self, df):
        
        """
        This function calculates distance between two points on a dataframe
        
        """

       
        coords_1 = (df['latitude_origin'], df['longitude_origin'])
        coords_2 = (df['latitude_destination'], df['longitude_destination'])

        return geopy.distance.vincenty(coords_1, coords_2).km


    
    

# class is initializing a dataframe 1 and dataframe 2
class pipeline(helperFunctions):
    
    # add a date of when all was scored
    def __init__(self, trips, airport, date):
        self.trips = trips.dropna()      #dropna
        self.airport = airport #.dropna()  #dropna
        self.date = date
    
        # initial join pre-processing
        # joined is necessary for most downstram stuff - gets me country groupings
        self.joined = super().group_region_country(self.trips, self.airport)
        
        self.accumulator = self.joined


        
    
    def group_price_bands(self):


        """
        This function groups various price columns into percentile tiers
        
        """
        
        dictionary = {'forecast_first_target_price':5, 'forecast_first_good_price':5, 'forecast_last_target_price':5, \
                     'forecast_last_good_price':5, 'forecast_min_target_price':5, 'forecast_max_target_price':5, \
                      'forecast_min_good_price':5, 'forecast_max_good_price':5}

        # iteration which changes the accumulator var (already initialized in class)
        for key, value in dictionary.items():

        
            self.accumulator[key + '_tier'] = super().banding(df=self.accumulator, categories=value, colName=key)


        return self.accumulator

    
    
    def group_distance_bands(self):
        
        """
        This function first calculates distances between destination and origin
        and then groups distances into tiers / bands
        
        """
        
        distances = self.accumulator.apply(super().distance, axis=1)

        #join in to original df on indeces
        self.accumulator['distances'] = distances

        #now do banding of distances
        self.accumulator['distances_tier'] = super().banding(df=self.accumulator, categories=5, colName='distances')
     
        return self.accumulator
    
    
    def group_trip_duration(self):

        """
        This function groups trips by duration

        """


        #now do banding of distances
        self.accumulator['duration_tier'] = super().banding(df=self.accumulator, categories=5, colName='stay')

        return self.accumulator




    def group_latitudes(self):
                

        """
        This function groups destination latitudes into climate zones. 0 to +/-30 is tropical, 
        +-30 to +-60 is continental, else is polar
        
        """


        # set up conditionals for latitude bands
        conditions = [
            (self.accumulator['latitude_destination'] > 0) & (self.accumulator['latitude_destination'] <= 30),
            (self.accumulator['latitude_destination'] < 0) & (self.accumulator['latitude_destination'] >= -30),
            (self.accumulator['latitude_destination'] > 30) & (self.accumulator['latitude_destination'] <= 60),
            (self.accumulator['latitude_destination'] < -30) & (self.accumulator['latitude_destination'] >= -60),
            (self.accumulator['latitude_destination'] > 60),
            (self.accumulator['latitude_destination'] < -60)]


        choices = ['tropical', 'tropical', 'continental', 'continental', 'polar', 'polar']

        
     
        # apply the choices
        self.accumulator['climate_destination'] = np.select(conditions, choices, default='black')

      
        return self.accumulator

    
    
    def group_dates(self):
        
        """
        This function groups dates
        
        """
        
        
        # EXTRACT MONTH_YEAR
        self.accumulator['month_year'] = self.accumulator['departure_date'].astype(str).str[0:7]

        #EXTRACT MONTH
        self.accumulator['month'] = self.accumulator['departure_date'].astype(str).str[5:7]

        return self.accumulator
    

    
    
    
    def price_clustering(self):
        
        """
        This function performs pre-processing, normalization, scaling and KMeans clustering
        on multiple pricing features to generate pricing clusters
        
        """


        prices_selected = self.accumulator[['lowest_price', 'forecast_first_good_price',   \
        'forecast_last_good_price', 'forecast_min_good_price', 'forecast_max_good_price']]

        prices_normalized = self.accumulator[['lowest_price', 'forecast_first_good_price',   \
        'forecast_last_good_price', 'forecast_min_good_price', 'forecast_max_good_price']]

        # unskew variables
        for col in prices_normalized.columns:
            prices_normalized[col] = np.log(prices_normalized[col])


        # import library 
        scaler = StandardScaler()

        #fit the scaler
        scaler.fit(prices_normalized)

        # transform
        prices_normalized_scaled = scaler.transform(prices_normalized)

        #reset index and col names
        prices_normalized_scaled_reset = pd.DataFrame(prices_normalized_scaled, \
                                        index=prices_normalized.index, \
                                        columns=prices_normalized.columns)


        # set the model
        kmeans = KMeans(n_clusters=8, random_state=1)

        # fit the model 
        kmeans.fit(prices_normalized_scaled_reset)

        cluster_labels = kmeans.labels_

        # #what is assignment
        prices_normalized_scaled_reset_labelled = prices_normalized_scaled_reset.assign(Cluster = cluster_labels)
        #prices_normalized_scaled_reset_labelled.head()

        self.accumulator = self.accumulator.assign(Cluster = cluster_labels) \
                           .rename(columns={'Cluster': 'Price_Cluster'})
        
        return self.accumulator

    
    


## Below is the run_pipeline() function interface which looks at "actions" user selected and modifies the dataset according to those actions. At the end you get a dataset with all the groupings and clusterings as new columns

In [3]:
%%time

#interface for pipeline
def run_pipeline(trips, airport, actions):
    
    """
    This function runs various steps of pipeline depending on actions
    that user has specified

    """
    
    today = date.today().strftime("%Y-%m-%d")
    
   
    # just chain the methods
    pipe = pipeline(trips, airport, today)


    if 'price_group' in actions:
        
        pipe.group_price_bands()
        
    
    if 'dates_group' in actions:
        
        pipe.group_dates()
    
    if 'latitude_group' in actions:
        
        pipe.group_latitudes()
    
    if 'distance_group' in actions:
        
        pipe.group_distance_bands()
    
    
    if 'price_clustering' in actions:
        
        pipe.price_clustering()
        
    if 'trip_duration_group' in actions:
        
        pipe.group_trip_duration()
    
    
    df_returned = pipe.accumulator
    df_returned['date_scored'] = today
    
    return df_returned






# import data
trips = pd.read_csv("./watch_sample.csv").drop(columns=['user_id']) 
airport = pd.read_csv("./airports.csv") 

#options for actions:
actions = ['price_group', 'dates_group', 'latitude_group', 'distance_group', 'price_clustering', 'trip_duration_group'] #

final_result = run_pipeline(trips, airport, actions)

final_result.head()

/Users/Don/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:85: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
/Users/Don/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:236: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 1min 30s, sys: 22.3 s, total: 1min 53s
Wall time: 2min 52s


,trip_id,trip_type,origin,destination,origin_type,destination_type,departure_date,return_date,stay,weekend,filter_no_lcc,filter_non_stop,filter_short_layover,filter_name,first_searched_dt,watch_added_dt,status_latest,latest_status_change_dt,status_updates,total_notifs,total_buy_notifs,first_rec,first_price,first_buy_dt,first_buy_price,lowest_price,last_notif_dt,forecast_first_target_price,forecast_first_good_price,forecast_last_target_price,forecast_last_good_price,forecast_last_warning_date,forecast_last_danger_date,forecast_min_target_price,forecast_max_target_price,forecast_min_good_price,forecast_max_good_price,first_advance,watch_advance,location_id_origin,location_type_origin,airport_group_origin,country_code_origin,country_origin,region_origin,latitude_origin,longitude_origin,location_id_destination,location_type_destination,airport_group_destination,country_code_destination,country_destination,region_destination,latitude_destination,longitude_destination,forecast_first_target_price_tier,forecast_first_good_price_tier,forecast_last_target_price_tier,forecast_last_good_price_tier,forecast_min_target_price_tier,forecast_max_target_price_tier,forecast_min_good_price_tier,forecast_max_good_price_tier,month_year,month,climate_destination,distances,distances_tier,Price_Cluster,duration_tier,date_scored
0,8bf4507a-99c4-3b45-9e17-ab7793a1ebb9,round_trip,airport/SJO,airport/JFK,airport,airport,2018-09-27,2018-10-04,7.0,0,0,0,1,ShortLayover,2018-05-09 13:09:04,2018-05-09 13:09:05,inactive,2018-05-25 18:25:53,5,2.0,1.0,wait,488.0,2018-05-16 14:50:35,416.0,392.0,2018-05-20 03:35:22,457.0,394.0,411.0,393.0,2018-05-26,2018-09-06,411.0,463.0,393.0,394.0,141,141.0,airport/SJO,airport,city/SJO,CR,Costa Rica,Mexico and Central America,9.993860,-84.208801,airport/JFK,airport,city/NYC,US,United States,United States,40.639801,-73.778900,3,3,2,3,3,2,3,3,2018-09,09,continental,3547.753921,3,6,3,2020-01-15
1,70929787-204d-3660-a6f1-c86b54bfc3a8,round_trip,airport/LAX,airport/FLL,airport,airport,2018-06-20,2018-06-26,6.0,0,0,0,0,NoFilter,2018-05-09 12:19:38,2018-05-09 12:19:59,inactive,2018-05-26 23:40:59,3,5.0,4.0,wait,324.0,2018-05-14 21:21:14,322.0,274.0,2018-05-22 13:40:19,301.0,275.0,291.0,275.0,2018-05-26,2018-06-12,279.0,310.0,275.0,275.0,42,42.0,airport/LAX,airport,airport/LAX,US,United States,United States,33.942501,-118.407997,airport/FLL,airport,airport/FLL,US,United States,United States,26.072599,-80.152702,2,2,1,2,2,1,2,2,2018-06,06,tropical,3769.572294,3,3,3,2020-01-15
2,12eb97df-08e8-3888-acda-9529e23b5227,round_trip,airport/ORD,airport/PVR,airport,airport,2018-02-16,2018-02-20,4.0,0,0,0,0,NoFilter,2018-01-06 12:52:11,2018-01-06 12:52:16,inactive,2018-01-31 20:10:09,3,4.0,2.0,wait,453.0,2018-01-18 23:28:00,355.0,355.0,2018-01-30 09:41:35,366.0,340.0,404.0,341.0,2018-01-30,2018-02-15,338.0,404.0,340.0,341.0,41,41.0,airport/ORD,airport,city/CHI,US,United States,United States,41.978600,-87.904800,airport/PVR,airport,airport/PVR,MX,Mexico,Mexico and Central America,20.680099,-105.253998,3,3,2,3,3,2,3,3,2018-02,02,tropical,2868.984524,3,6,2,2020-01-15
3,b0513720-e227-3298-8e6b-c0f98be02674,round_trip,city/QHO,airport/MTY,city,airport,2018-02-22,2018-02-26,4.0,1,0,1,0,NonStop,2018-02-17 22:01:33,2018-02-17 22:01:35,expired,2018-02-23 03:00:00,3,1.0,1.0,buy,465.0,2018-02-17 22:01:33,465.0,465.0,2018-02-22 08:52:28,275.0,179.0,472.0,179.0,2018-02-10,2018-02-16,275.0,472.0,179.0,179.0,5,5.0,city/QHO,city,city/QHO,US,United States,United States,29.814899,-95.310150,airport/MTY,airport,airport/MTY,MX,Mexico,Mexico and Central America,25.778500,-100.107002,2,1,2,1,2,2,1,1,2018-02,02,tropical,650.640923,1,4,2,2020-01-15
4,af0e1df5-77fb-32bf-9760-586528f40f0b,round_trip,airport/MSP,airport/ELP,airport,airport,2019-03-22,2019-03-30,8.0,0,0,0,0,NoFilter,2019-01-09 13:35:50,2019-01-09 13:36:07,inactive,2019-02-02 07:42:22,3,3.0,3.0,buy,638.0,2019-01-09 13:35:50,638.0,616.0,2019-01-29 13:31:53,583.0,347.0,650.0,352.0,2018-08-23,2018-12-16,583.0,65

## This is simple prototype of a recommender we could run on each search. The result of recommender() function is a column of recommended trip_ids based on the groupings a given search has.

In [4]:
%%time


def recommender(df, df_searchable):
    
    """
    This function is a prototype of a recommender. It takes in a dataframe df
    which contains trips that we want to fetch recommendations for. To do that,
    we extract relevant grouping/cluster fields from df and then search which
    trips haev same grouping values in the df_searchable

    """
    
    # non-negotiable
    month_year = df['month_year']
    country_origin = df['country_origin']
    trip_type = df['trip_type']
    
    #geographical features
    region_destination = df['region_destination']
    climate_destination = df['climate_destination']
    
    #price features
    Price_Cluster = df['Price_Cluster']
    
    # duration feature
    duration_tier = df['duration_tier']
    
    
    # filtering
    df_output = df_searchable[(df_searchable['month_year'] == month_year) & (df_searchable['country_origin'] == country_origin ) \
             & (df_searchable['trip_type'] == trip_type ) & (df_searchable['region_destination'] == region_destination ) \
              & (df_searchable['climate_destination'] == climate_destination )  & (df_searchable['Price_Cluster'] == Price_Cluster ) \
               & (df_searchable['duration_tier'] == duration_tier)]
    
 
    
    list_returned = df_output['trip_id'].tolist()  #limit for now
    
    df['recommendations'] = list_returned[0:5]
    df['num_of_recommendations'] = len(list_returned)
    
    return df
    
    

    
    
# dataset to search - remove any NAs for critical columns. THESE ARE SEARCHES WE WANT TO GET RECOS FOR
query_df = final_result.dropna().head()


# pass row in - get the calculation of recommendations
recommendations = query_df.apply(recommender, df_searchable = final_result, axis=1)

recommendations.head()

CPU times: user 1.52 s, sys: 287 ms, total: 1.8 s
Wall time: 1.93 s


,trip_id,trip_type,origin,destination,origin_type,destination_type,departure_date,return_date,stay,weekend,filter_no_lcc,filter_non_stop,filter_short_layover,filter_name,first_searched_dt,watch_added_dt,status_latest,latest_status_change_dt,status_updates,total_notifs,total_buy_notifs,first_rec,first_price,first_buy_dt,first_buy_price,lowest_price,last_notif_dt,forecast_first_target_price,forecast_first_good_price,forecast_last_target_price,forecast_last_good_price,forecast_last_warning_date,forecast_last_danger_date,forecast_min_target_price,forecast_max_target_price,forecast_min_good_price,forecast_max_good_price,first_advance,watch_advance,location_id_origin,location_type_origin,airport_group_origin,country_code_origin,country_origin,region_origin,latitude_origin,longitude_origin,location_id_destination,location_type_destination,airport_group_destination,country_code_destination,country_destination,region_destination,latitude_destination,longitude_destination,forecast_first_target_price_tier,forecast_first_good_price_tier,forecast_last_target_price_tier,forecast_last_good_price_tier,forecast_min_target_price_tier,forecast_max_target_price_tier,forecast_min_good_price_tier,forecast_max_good_price_tier,month_year,month,climate_destination,distances,distances_tier,Price_Cluster,duration_tier,date_scored,recommendations,num_of_recommendations
0,8bf4507a-99c4-3b45-9e17-ab7793a1ebb9,round_trip,airport/SJO,airport/JFK,airport,airport,2018-09-27,2018-10-04,7.0,0,0,0,1,ShortLayover,2018-05-09 13:09:04,2018-05-09 13:09:05,inactive,2018-05-25 18:25:53,5,2.0,1.0,wait,488.0,2018-05-16 14:50:35,416.0,392.0,2018-05-20 03:35:22,457.0,394.0,411.0,393.0,2018-05-26,2018-09-06,411.0,463.0,393.0,394.0,141,141.0,airport/SJO,airport,city/SJO,CR,Costa Rica,Mexico and Central America,9.993860,-84.208801,airport/JFK,airport,city/NYC,US,United States,United States,40.639801,-73.778900,3,3,2,3,3,2,3,3,2018-09,09,continental,3547.753921,3,6,3,2020-01-15,"[8bf4507a-99c4-3b45-9e17-ab7793a1ebb9, 5761ffd...",8
1,70929787-204d-3660-a6f1-c86b54bfc3a8,round_trip,airport/LAX,airport/FLL,airport,airport,2018-06-20,2018-06-26,6.0,0,0,0,0,NoFilter,2018-05-09 12:19:38,2018-05-09 12:19:59,inactive,2018-05-26 23:40:59,3,5.0,4.0,wait,324.0,2018-05-14 21:21:14,322.0,274.0,2018-05-22 13:40:19,301.0,275.0,291.0,275.0,2018-05-26,2018-06-12,279.0,310.0,275.0,275.0,42,42.0,airport/LAX,airport,airport/LAX,US,United States,United States,33.942501,-118.407997,airport/FLL,airport,airport/FLL,US,United States,United States,26.072599,-80.152702,2,2,1,2,2,1,2,2,2018-06,06,tropical,3769.572294,3,3,3,2020-01-15,"[70929787-204d-3660-a6f1-c86b54bfc3a8, 51f7ad1...",96
2,12eb97df-08e8-3888-acda-9529e23b5227,round_trip,airport/ORD,airport/PVR,airport,airport,2018-02-16,2018-02-20,4.0,0,0,0,0,NoFilter,2018-01-06 12:52:11,2018-01-06 12:52:16,inactive,2018-01-31 20:10:09,3,4.0,2.0,wait,453.0,2018-01-18 23:28:00,355.0,355.0,2018-01-30 09:41:35,366.0,340.0,404.0,341.0,2018-01-30,2018-02-15,338.0,404.0,340.0,341.0,41,41.0,airport/ORD,airport,city/CHI,US,United States,United States,41.978600,-87.904800,airport/PVR,airport,airport/PVR,MX,Mexico,Mexico and Central America,20.680099,-105.253998,3,3,2,3,3,2,3,3,2018-02,02,tropical,2868.984524,3,6,2,2020-01-15,"[12eb97df-08e8-3888-acda-9529e23b5227, a474146...",45
3,b0513720-e227-3298-8e6b-c0f98be02674,round_trip,city/QHO,airport/MTY,city,airport,2018-02-22,2018-02-26,4.0,1,0,1,0,NonStop,2018-02-17 22:01:33,2018-02-17 22:01:35,expired,2018-02-23 03:00:00,3,1.0,1.0,buy,465.0,2018-02-17 22:01:33,465.0,465.0,2018-02-22 08:52:28,275.0,179.0,472.0,179.0,2018-02-10,2018-02-16,275.0,472.0,179.0,179.0,5,5.0,city/QHO,city,city/QHO,US,United States,United States,29.814899,-95.310150,airport/MTY,airport,airport/MTY,MX,Mexico,Mexico and Central America,25.778500,-100.107002,2,1,2,1,2,2,1,1,2018-02,02,tropical,650.640923,1,4,2,2020-01-15,"[b0513720-e227-3298-8e6b-c0f98be02674, 1c29ffc...",4
4,af0e1df5-77fb-32bf-9760-586528f40f0b,round_trip,airport/MSP,airport/E